# Build new PDFF phenotype file

In [93]:
import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [107]:
xmdu_old_data = pd.read_table('/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/Xiaomeng/SAIGE/origdata/PDFF_4_7/pheno_pc_age_sex.allwhite.PDFF_4_7.txt')
print(xmdu_old_data.shape)

(43293, 18)


In [108]:
new_pdff = pd.read_table('/mnt/speliotes-lab/Projects/UK_ATLAS/Liver_MRI/PDFF/data/UKBB_liver_MRI_New_GRE_and_New_IDEAL_PDFF_combine.fillmissing.tab.2023.txt')
print(new_pdff.shape)
new_pdff

(502396, 4)


,eid,ture_PDFF_22436_24352,true_PDFF_21088_21092,true_PDFF_40061
0,1000013,NaN,NaN,NaN
1,1000024,NaN,NaN,NaN
2,1000036,NaN,NaN,NaN
3,1000048,NaN,NaN,NaN
4,1000055,NaN,NaN,NaN
...,...,...,...,...
502391,6026172,NaN,NaN,NaN
502392,6026180,NaN,NaN,NaN
502393,6026199,NaN,NaN,NaN
502394,6026205,NaN,NaN,NaN


In [109]:
eid2new_pdff = {int(e):p for e,p in zip(new_pdff.eid, new_pdff.true_PDFF_40061) if not np.isnan(p)}
print(len(eid2new_pdff))
xmdu_old_data["PDFF_new_real"] = [eid2new_pdff[int(e)] if int(e) in eid2new_pdff else np.nan for e in xmdu_old_data.eid]
xmdu_old_data

40533


,eid,sex,age,age2,white,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,PDFF,PDFF_INT,y,PDFF_new_real
0,1000225,0,76,5776,1.0,-11.5012,4.71634,-0.909552,-3.228350,-8.70575,-1.109500,1.760170,-2.505820,-1.041380,-3.401380,1.291345,-0.804387,-0.804392,3.1
1,1000351,1,59,3481,1.0,-12.9634,2.77750,-1.361070,1.917940,-4.06609,-2.352210,1.935410,-0.930296,4.096230,1.401510,3.095536,0.336321,0.336323,3.0
2,1000687,1,71,5041,1.0,-10.9044,3.65342,-1.781790,-2.295460,-6.64868,-2.079730,-1.024310,-1.878760,-0.372462,4.454860,3.747685,0.518313,0.518316,2.6
3,1000845,0,60,3600,1.0,-12.6761,4.21835,-1.617890,-3.790970,-6.93775,-3.595830,5.083040,-2.390520,1.916030,0.195406,2.386000,0.080448,0.080449,3.0
4,1000897,1,57,3249,1.0,-13.0404,3.50887,-2.534000,4.740730,7.98885,-2.205200,-0.666430,-0.683902,-0.003372,0.243744,1.742808,-0.271369,-0.271370,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43288,6025250,1,66,4356,1.0,-11.0232,4.40977,-0.684932,0.687638,8.21459,-0.272046,1.507370,1.936180,5.778630,1.132180,1.123043,-1.299170,-1.299184,2.5
43289,6025976,1,73,5329,1.0,-10.1363,4.08336,0.379782,-1.158300,-7.97296,1.956960,-0.433572,-1.759680,2.203440,-0.825871,1.147706,-1.200618,-1.200630,1.9
43290,6026006,0,62,3844,1.0,-13.9992,4.27860,-4.739770,-0.814331,-11.25030,-2.150080,1.547940,-1.925550,6.103740,0.900324,11.146381,1.486074,1.486093,13.2
43291,6026014,1,72,5184,NaN,10.5725,-3.88749,14.568100,-36.948400,4.45948,3.719610,20.886300,10.737700,-1.648670,-10.953900,2.392979,0.083232,0.083232,3.5


In [110]:
sum([e not in eid2new_pdff for e in xmdu_old_data.eid])

3815

Seems like 10 people from the earlier PDFF analysis dropped out

In [111]:
xmdu_new_pdff_real_data = xmdu_old_data[[not np.isnan(p) for p in xmdu_old_data.PDFF_new_real]]
print(xmdu_new_pdff_real_data.shape)

(39478, 19)


In [20]:
#xmdu_new_pdff_real_data.to_csv('/home/craut/wkspce/new_MRI_PDFF/data_out/xmdu_new_pdff_real_results/new_pdff_real_data.txt', index=False,sep='\t')

# Evaluate the efficacy of the old predicted PDFF values against ground truth

In [21]:
# calculate RMSE for PDFF_new_real vs PDFF
print(np.sqrt(np.mean((xmdu_new_pdff_real_data.PDFF_new_real - xmdu_new_pdff_real_data.PDFF)**2)))

2.090493159306228


In [23]:
# grab the list of individuals imaged through the gre protocol
gre_list = pd.read_table("/mnt/speliotes-lab/Projects/UK_ATLAS/Liver_MRI/GRE/segmentation/data/GRE_id.list",header=None)
gre_list

,0
0,1000845
1,1000897
2,1001130
3,1001429
4,1001516
...,...
10100,6020951
10101,6022215
10102,6023521
10103,6023722


In [24]:
gre_eid_set = set(gre_list[0])
print(len(gre_eid_set))

10105


In [25]:
gre_pat = xmdu_new_pdff_real_data[[e in gre_eid_set for e in xmdu_new_pdff_real_data.eid]]
gre_pat

,eid,sex,age,age2,white,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,PDFF,PDFF_INT,y,PDFF_new_real
3,1000845,0,60,3600,1.0,-12.6761,4.218350,-1.617890,-3.790970,-6.937750,-3.595830,5.083040,-2.390520,1.916030,0.195406,2.386000,0.080448,0.080449,3.0
4,1000897,1,57,3249,1.0,-13.0404,3.508870,-2.534000,4.740730,7.988850,-2.205200,-0.666430,-0.683902,-0.003372,0.243744,1.742808,-0.271369,-0.271370,2.5
8,1001429,1,70,4900,1.0,-12.9070,5.362040,-0.762431,-1.445210,-6.396520,-0.045248,-0.561997,0.944649,3.268100,0.628948,2.261000,0.022975,0.022975,3.0
10,1001516,1,68,4624,1.0,-14.1055,1.411500,-0.062417,3.467960,-3.551750,-2.375860,-0.559764,-2.537620,-1.137310,3.955310,2.078488,-0.066830,-0.066830,2.5
12,1001571,1,58,3364,1.0,-10.3700,0.615862,0.051317,0.891460,-0.299649,-0.234438,-0.418748,-1.738240,1.310590,2.025910,2.815393,0.244600,0.244602,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43250,6020951,1,49,2401,1.0,-12.3399,2.425260,1.960130,-0.644627,-0.441872,1.258760,1.577970,0.853969,-0.052397,0.847384,2.005000,-0.106309,-0.106309,2.4
43257,6022215,1,48,2304,NaN,401.5190,77.535000,-8.024950,6.085320,-0.242304,0.132989,1.993590,0.308730,-0.668240,-1.138160,1.160000,-1.158215,-1.158225,1.7
43272,6023521,0,62,3844,1.0,-11.4574,5.343780,-1.208180,0.154604,-7.410120,-0.485990,-0.486639,-0.926907,2.779360,3.559500,1.185000,-1.071298,-1.071308,1.5
43274,6023722,0,55,3025,1.0,-14.2690,3.383110,-5.475330,2.972100,-3.580540,0.831593,-2.415960,-2.720150,3.874360,5.268320,1.529000,-0.455227,-0.455230,2.0


In [26]:
# rmse for gre protocol only
print(np.sqrt(np.mean((gre_pat.PDFF_new_real - gre_pat.PDFF)**2)))

1.2265421230253637


In [27]:
# filter anyone who was not gre --> ideal protocol
ideal_pat = xmdu_new_pdff_real_data[[e not in gre_eid_set for e in xmdu_new_pdff_real_data.eid]]
ideal_pat

,eid,sex,age,age2,white,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,PDFF,PDFF_INT,y,PDFF_new_real
0,1000225,0,76,5776,1.0,-11.5012,4.71634,-0.909552,-3.228350,-8.70575,-1.109500,1.760170,-2.505820,-1.041380,-3.401380,1.291345,-0.804387,-0.804392,3.1
1,1000351,1,59,3481,1.0,-12.9634,2.77750,-1.361070,1.917940,-4.06609,-2.352210,1.935410,-0.930296,4.096230,1.401510,3.095536,0.336321,0.336323,3.0
2,1000687,1,71,5041,1.0,-10.9044,3.65342,-1.781790,-2.295460,-6.64868,-2.079730,-1.024310,-1.878760,-0.372462,4.454860,3.747685,0.518313,0.518316,2.6
5,1000908,1,60,3600,1.0,-12.3835,5.67476,-1.608530,3.150930,3.24340,-0.356158,-0.964057,1.823540,2.475230,-0.006745,5.089967,0.805505,0.805511,6.4
6,1001022,0,69,4761,NaN,3.1045,7.48312,-2.646800,5.063720,4.52070,-2.483930,4.277650,-1.633850,-7.591340,3.666880,1.757694,-0.258141,-0.258142,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43288,6025250,1,66,4356,1.0,-11.0232,4.40977,-0.684932,0.687638,8.21459,-0.272046,1.507370,1.936180,5.778630,1.132180,1.123043,-1.299170,-1.299184,2.5
43289,6025976,1,73,5329,1.0,-10.1363,4.08336,0.379782,-1.158300,-7.97296,1.956960,-0.433572,-1.759680,2.203440,-0.825871,1.147706,-1.200618,-1.200630,1.9
43290,6026006,0,62,3844,1.0,-13.9992,4.27860,-4.739770,-0.814331,-11.25030,-2.150080,1.547940,-1.925550,6.103740,0.900324,11.146381,1.486074,1.486093,13.2
43291,6026014,1,72,5184,NaN,10.5725,-3.88749,14.568100,-36.948400,4.45948,3.719610,20.886300,10.737700,-1.648670,-10.953900,2.392979,0.083232,0.083232,3.5


In [28]:
# rmse for ideal protocol
print(np.sqrt(np.mean((ideal_pat.PDFF_new_real - ideal_pat.PDFF)**2)))

2.270272049486994


In [71]:
def make_scatter(true,pred,title="",color=None,eid=None,symbol=None):
    true = np.array(true)
    pred = np.array(pred)
    rmse = np.sqrt(np.mean((true-pred)**2))
    pearson_correlation = np.corrcoef(true,pred)[0,1]
    title = f"{title} (N={len(true)}, RMSE={rmse:.2f}, Pearson={pearson_correlation:.2f})"
    if not color is None:
        color = np.array([str(c) for c in color])
    fig = px.scatter(x=true,y=pred,color=color,title=title,opacity=0.5,hover_name=[f"eid:{e}" for e in eid],symbol=symbol)
    ylim = min(max(pred),max(true))
    fig.add_trace(px.line(x=[0, ylim], y=[0, ylim]).data[0])
    fig.update_traces(line=dict(dash="dash",color="black"))
    fig.update_layout(xaxis_title="True PDFF (%)", yaxis_title="ResNet Precited PDFF (%)")
    return fig

In [79]:
df = gre_pat
fig = make_scatter(df.PDFF,
                df.PDFF_new_real,
                "GRE true vs predicted PDFF scatterplot",
                ["white" if not np.isnan(w) else "not white" for w in df.white],
                df.eid,
                ["Male" if s == 1 else "Female" for s in df.sex])
fig.write_html("gre_true_vs_pred.html")

In [80]:
df = ideal_pat
fig = make_scatter(df.PDFF,
             df.PDFF_new_real,
             "IDEAL true vs predicted PDFF scatterplot",
             ["white" if not np.isnan(w) else "not white" for w in df.white],
             df.eid,
             ["Male" if s == 1 else "Female" for s in df.sex])
fig.write_html("IDEAL_true_vs_pred.html")

In [81]:
df = xmdu_new_pdff_real_data
fig = make_scatter(df.PDFF,
             df.PDFF_new_real,
             "GRE & IDEAL true vs predicted PDFF scatterplot",
             ["GRE" if int(e) in gre_eid_set else "IDEAL" for e in df.eid],
             df.eid,
             ["Male" if s == 1 else "Female" for s in df.sex])
fig.write_html("both_true_vs_pred.html")

# Test for Sex and White bias

We know that the UKBB is a predominantly white patient population. There should also be a pretty good balance of across the sex's. I just want to make sure that there is no major bias in the predictions made earlier which we can evaluate now due to the discovery of the true PDFF values.

In [119]:
# correct a few columns
updated_df = pd.DataFrame(xmdu_new_pdff_real_data)
updated_df["protocol"] = ["GRE" if int(e) in gre_eid_set else "IDEAL" for e in updated_df.eid]
updated_df["white_labeled"] = ["white" if not np.isnan(w) else "not_white" for w in xmdu_new_pdff_real_data.white]
updated_df["white_labeled"] = pd.Categorical(updated_df["white_labeled"], categories=["white","not_white"])
updated_df["sex_labeled"] = ["Male" if s == 1 else "Female" for s in xmdu_new_pdff_real_data.sex]
updated_df.sort_values(by=["protocol","sex_labeled","white_labeled"], ascending=[True,True,False],inplace=True)
updated_df.reset_index(drop=True,inplace=True)


# build 2-way table to establish counts of whites and sex 
df = updated_df
table = pd.crosstab([df["protocol"], df["white_labeled"]],df["sex_labeled"], margins=True, margins_name="Total")
table

sex_labeled             Female   Male  Total
protocol white_labeled                      
GRE      white            3882   3561   7443
         not_white         529    508   1037
IDEAL    white           13904  12986  26890
         not_white        2233   1875   4108
Total                    20548  18930  39478

So after talking with Antonio I think we want to test the following hypothesis:
- Is there an effect on error with the following?
  - sex
  - white
  - protocol
  - age

We can test this through the below model:
$$\text{true}-\text{predicted} = \beta_0 + \beta_1 \text{sex} + \beta_2 \text{white} + \beta_3 \text{protocol} + \beta_4 \text{age} + \epsilon$$
$$H_0: \beta_0 = \beta_1 = \beta_2 = \beta_3 = \beta_4$$
$$H_1: \text{any } \beta \neq 0$$



In [115]:
df

,eid,sex,age,age2,white,pc1,pc2,pc3,pc4,pc5,...,pc8,pc9,pc10,PDFF,PDFF_INT,y,PDFF_new_real,protocol,white_labeled,sex_labeled
0,1000845,0,60,3600,1.0,-12.67610,4.218350,-1.617890,-3.790970,-6.937750,...,-2.390520,1.916030,0.195406,2.386000,0.080448,0.080449,3.0,GRE,white,Female
1,1002117,0,46,2116,1.0,-10.36850,3.308830,0.048704,1.904190,-7.841710,...,1.737460,-1.224340,0.622513,0.984000,-2.129575,-2.129642,1.3,GRE,white,Female
2,1003297,0,58,3364,1.0,-13.52230,5.772310,-3.590790,1.886390,3.155850,...,-0.027227,-2.487850,-3.131280,1.147000,-1.202224,-1.202235,1.9,GRE,white,Female
3,1004445,0,59,3481,1.0,-10.25450,3.408410,0.565607,-1.682310,-4.978340,...,0.848904,2.908500,-4.152770,2.138000,-0.035117,-0.035117,2.5,GRE,white,Female
4,1004476,0,70,4900,1.0,-10.78690,1.822760,0.873310,0.270541,0.593214,...,-0.187681,-11.832300,4.147410,1.558000,-0.426628,-0.426630,2.0,GRE,white,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39473,6004226,1,66,4356,NaN,10.98960,-2.137960,15.806100,-35.458800,-0.293184,...,6.034630,3.147420,-1.371230,3.343385,0.410635,0.410638,4.0,IDEAL,not_white,Male
39474,6005539,1,58,3364,NaN,1.81694,-4.267670,12.366200,-30.047800,-0.735583,...,22.570400,-4.187500,-6.770600,2.888783,0.271129,0.271130,4.3,IDEAL,not_white,Male
39475,6016063,1,75,5625,NaN,-1.80040,0.501068,2.542080,-21.407900,8.812690,...,-0.988631,2.184550,-0.192609,5.228311,0.828828,0.828834,6.8,IDEAL,not_white,Male
39476,6016074,1,70,4900,NaN,-12.70570,4.311340,-3.692600,10.680300,26.433600,...,0.364135,-0.199044,1.332840,9.506418,1.347871,1.347885,7.9,IDEAL,not_white,Male


In [120]:
y = np.array(df.PDFF_new_real) - np.array(df.PDFF)
df["y"] = y
mod = smf.ols(formula='y ~ sex_labeled + white_labeled + protocol + age', data=df)

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     100.6
Date:                Thu, 03 Aug 2023   Prob (F-statistic):           2.43e-85
Time:                        18:54:37   Log-Likelihood:                -78235.
No. Observations:               39478   AIC:                         1.565e+05
Df Residuals:                   39473   BIC:                         1.565e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

We can see that there is a statistically significant association with sex and age with bias but not ancestry.